In [1]:
import urllib2
import random
from math import exp,fabs,log10
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *
import random

In [ ]:
## Computing non linear representation of the data

def define_Z(X,d):
    poly = PolynomialFeatures(d)
    Z = poly.fit_transform(X)
    return Z

In [ ]:
##computes the accuracy of the predicted value
def accuracy(Y,Y_hat):
                
    sum1 = sum([1 for i in range(len(Y)) if Y[i] == Y_hat[i]])
    return (1.0*sum1/len(Y))*100

In [ ]:
## return a value betwween 0 and 1

def sigmoid_2class(theta,z):
    z = np.reshape(np.array(z),(len(z),1))
    return (1/(1 + exp(-np.dot(np.transpose(theta),z))))

In [ ]:
## traning method for 2 class clssification
def training_2class(Z,Y,lr):
    
    best = False    
    theta = np.zeros(shape=(Z.shape[1],1))
    theta.fill(0.01)
    max_it = 0    
    l_new = -1.0 * sum([(Y[i]*log10(sigmoid_2class(Z[i],theta))) + ((1-Y[i])*log10(1-sigmoid_2class(Z[i],theta))) for i in range(Z.shape[0])])
    while best == False:
        
        l_pre = l_new
        
        
        grad = 0
        
        for i in range(Z.shape[0]):
            
            grad += (sigmoid_2class(theta,Z[i]) - Y[i]) * np.reshape(np.array(Z[i]),(len(Z[i]),1))
        
        theta = theta - lr*grad
        
        l_new = -1.0 * sum([(Y[i]*log10(sigmoid_2class(Z[i],theta))) + ((1-Y[i])*log10(1-sigmoid_2class(Z[i],theta))) for i in range(Z.shape[0])])
        
        #print l_new
               
        max_it += 1
        #print max_it
        if max_it == 200 or abs(l_new - l_pre) < 0.2 :
            best = True
            
    return theta
    

In [ ]:
## using ten cross fold to compute the error

def ten_cross_fold_2class(Z,Y,lr):    
    
    
    accuracies = []
    cv = KFold(len(Y),10)
    for train_id,test_id in cv:
        Y_hat = []
        theta = training_2class(Z[train_id],Y[train_id],lr)
        
        Y_hat = [1 if sigmoid_2class(theta,z) > 0.5 else 0 for z in Z[test_id]]
        
        accuracies.append(accuracy(Y[test_id],Y_hat))
        
    return np.mean(accuracies)

In [ ]:
## using the bank note data set for 2 class classification

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
data = urllib2.urlopen(url)
data = data.read()
data = data.split("\r\n")
random.shuffle(data)

X = []
Y = []
for i in data:
    i = i.split(',')
    Y.append(int(i[len(i)-1]))
    i = map(float,i[0:len(i)-1])
    X.append(i)
    
X = np.array(X)
cat = (set(Y))
Y = np.reshape(np.array(Y),(len(Y),1))
#print X.shape
#print Y.shape
#print cat
Z = define_Z(X,2)
#print Z.shape

#theta = training_2class(Z[:1300],Y[:1300],.00001)
error = ten_cross_fold_2class(Z,Y,0.00001)
print error

In [ ]:
######################################################################################################################

In [ ]:
#### indication function to return 1 and O depending on the class

def indicator(y,k):
    if y==k:
        return 1.0
    else:
        return 0.0

In [ ]:
## computing the actual prediction


def classification_kclass(thetas,z,K):
    
    g = [softmax_kclass(thetas,z,k) for k in range(K)]
    return g.index(max(g))

In [ ]:
##returns a value between 0 and 1

def softmax_kclass(thetas,z,cls):
    
    z = np.reshape(np.array(z),(len(z),1))
    d = 0
    
    d = sum([ exp(np.dot(np.transpose(theta),z)) for theta in thetas])
    
    n = exp(np.dot(np.transpose(thetas[cls]),z))
    return (1.*n/d)

In [ ]:
# training function k class classifier

def training_kclass(Z,Y,lr,K):
    
    thetas = np.zeros(shape=(K,Z.shape[1]))
    
    thetas.fill(0.1)
    
    #print thetas
    
    max_it = 0
    
    best = False
    
    l_new = -1.0 * sum([((indicator(Y[i],k))*(log10(softmax_kclass(thetas,Z[i],k)))) for i in range(Z.shape[0]) for k in range(K)])
    
    while best == False:
        
        l_pre = l_new
        
        for k in range(K):

            grad = 0

            for i in range(Z.shape[0]):

                grad += (softmax_kclass(thetas,Z[i],k) - indicator(Y[i],k))*Z[i]
            
            thetas[k] = thetas[k] - lr*grad

        

        l_new = -1.0 * sum([((indicator(Y[i],k))*(log10(softmax_kclass(thetas,Z[i],k)))) for i in range(Z.shape[0]) for k in range(K)])

        
        
        max_it += 1
        
        if max_it == 200 or abs(l_new - l_pre) < 0.1 :
            best = True
            
    return thetas
    

In [ ]:
## using ten cross fold error to compute the error

def ten_cross_fold_kclass(Z,Y,lr,K):    
    
    
    accuracies = []
    cv = KFold(len(Y),10)
    for train_id,test_id in cv:
        Y_hat = []
        thetas = training_kclass(Z[train_id],Y[train_id],lr,K)
        
               
        Y_hat = [ classification_kclass(thetas,z,K) for z in Z[test_id]]
        
        accuracies.append(accuracy(Y[test_id],Y_hat))
    return np.mean(accuracies)

In [ ]:
########################### Data for logistic Regression k class discrimination #########################

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
data = urllib2.urlopen(url)
data = data.read()
data = data.split("\n")
data = data[0:len(data)-2]
random.shuffle(data)

X = []
Y = []
for i in data:
    i = i.split(',')
    Y.append(i[len(i)-1])
    i = map(float,i[0:len(i)-1])
    X.append(i)
for i in range(len(Y)):
    if Y[i] == 'Iris-setosa':
        Y[i] = 0
    elif Y[i] == 'Iris-versicolor':
        Y[i] = 1
    elif Y[i] == 'Iris-virginica':
        Y[i] = 2
X = np.array(X)
cat = (set(Y))
Y = np.reshape(np.array(Y),(len(Y),1))
Z = define_Z(X,2)
K = len(cat)


error = ten_cross_fold_kclass(Z,Y,0.00001,K)
print error